# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
import numpy as np
tstart = '2018-07-19T10:00:00'
tend = '2018-07-20T10:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



[#################################] 100% deltat.preds


../orbit_engine/lunar_200to201.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2018-07-19 00:38:40,2018-07-19 01:34:27
1,2018-07-19 02:15:19,2018-07-19 03:11:06
2,2018-07-19 03:51:58,2018-07-19 04:47:45
3,2018-07-19 05:28:36,2018-07-19 06:24:23
4,2018-07-19 07:05:15,2018-07-19 08:01:02


In [4]:
import nustar_planning
import imp
imp.reload(nustar_planning)


<module 'nustar_planning' from '/Users/bwgref/science/local/anaconda/lib/python3.5/site-packages/nustar_planning-0.2.dev73-py3.5.egg/nustar_planning/__init__.py'>

In [5]:
# 6 arcminutes maximum drift (default)

nustar_planning.lunar_planning.position_shift(orbits[0:1], parallax_correction=True,
               outfile='drift_test.txt',
               dt = 5*u.s,pad_time = 5*u.min)


[#################################] 100% deltat.preds
[#################################] 100% deltat.preds


In [6]:
from astropy.coordinates import SkyCoord

In [7]:
f = open('drift_test.txt', 'r')
times = []
sky_pos = []
for line in f:
    if line.startswith('A'):
        continue


    fields = line.split()
    this_time = datetime.strptime(fields[0], '%Y:%j:%H:%M:%S')

    times = np.append(times, this_time)
    
    this_skypos = SkyCoord(ra=fields[1], dec=fields[2], unit ='deg')
    sky_pos = np.append(sky_pos, this_skypos)
    
f.close()

In [8]:
good_exp = 0.
all_exp = 0.
for ind in range(len(times)):
    if ind == 0:
        continue
    dt = (times[ind] - times[ind-1]).total_seconds()
    dr = sky_pos[ind].separation(sky_pos[ind-1]).arcmin
    print(dt, dr)
    

170.0 6.028474052785369
335.0 6.036811039864043
495.0 6.031551494640764
175.0 6.044162340901583
140.0 6.166294683204976
125.0 6.194466513973696
115.0 6.282517793014867
110.0 6.224043354955086
105.0 6.043603968896318
100.0 6.070398058081941
100.0 6.011902363470457
100.0 6.15683610461505
105.0 6.177562048651007
110.0 6.048839328195584
115.0 6.196804473535783
130.0 6.127542732042249
150.0 6.045617561125026
195.0 6.078556337294491
770.0 6.13454311034621
165.0 6.01356478563543
